# GFK

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#adaptacion
from utils.adaptacion import gfk_grid_search, transform_gfk

#clasificadores
from utils.clasificacion import *

#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *

#otros
import os
import numpy as np
import pandas as pd
from sklearn.externals import joblib
import itertools

#variables para guardar los resultados
tipo = pruebas[2]
dataset_name = datasets[1]

Using Theano backend.
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


# Adaptación

## Creación de modelos de adaptación.

Por cada par de dominios se entrenan distintos modelos según los parámetros establecidos.
Al entrenar cada modelo, este es inmediatamente probado, siendo almacenado o descartado según su desempeño.

Cada modelo es guardado en la ruta: models/twitter/gfk/me2\_[dimensiones]\_[dominio\_fuente]_[dominio\_objetivo].pkl

# Pruebas con el dataset Twitter (2000 Dimensiones)

In [3]:
dims = 2000

In [4]:
print tipo
print dataset_name
print dims
print data_path

gfk
twitter
2000
data


In [5]:
# cargando dataset Twitter
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

Dataset already splitted


In [ ]:
models_paths = os.path.join(models_path, dataset_name, tipo, 'me2_%d_models_paths.pkl' % dims)

parameters = {
    'dims': [int(dims/4), int(dims/2)],
    'n_subs': [10, 20, 50]
}

pairs = list(itertools.permutations(domains, 2))

#por cada par se entrena un adaptador y se guarda el que mejor resultados entrega
print "Adaptando entre:"
for src, tgt in pairs:
    print "%s - %s" % (src, tgt)
    X_src = labeled[src]['X_tr'][:, :dims].todense()
    y_src = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)

    X_tgt = labeled[tgt]['X_tr'][:, :dims].todense()


    gfk, score = gfk_grid_search(X_src, y_src, X_tgt, parameters, n_jobs=4)
    print "\tMejor modelo: %.3f" % score

    folder_path = os.path.join(models_path, dataset_name, tipo)
    prefix = "me2_%d_%s_%s.pkl" % (dims, src, tgt)

    best_model_save_path = os.path.join(folder_path, prefix)
    print "\tGuardando modelo en %s" % best_model_save_path
    joblib.dump(gfk, best_model_save_path)

print "\nAdaptaciones completadas"

Adaptando entre:
rio2016 - thevoice
	n: 10 - d: 500

/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with Standar

 0.788850174216
	n: 20 - d: 500 0.865783972125
	n: 50 - d: 500 0.903275261324
	n: 10 - d: 1000 0.805853658537
	n: 20 - d: 1000 0.853797909408
	n: 50 - d: 1000

### Pruebas de clasificación

In [ ]:
df = pd.DataFrame(columns=dataframe_columns)

i=0
tareas = len(domains)*(len(domains)-1)
pairs = list(itertools.permutations(domains, 2))

# por cada par posible para adaptar
for src, tgt in pairs:
    print "Tarea %d de %d" % (i+1, tareas)
            
    #baseline in-domain error
    #e_b(T,T)
    #entrenado en dominio tgt y probado en dominio tgt
    X_tr = labeled[tgt]['X_tr'][:, :dims].todense()
    y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)

    X_ts = labeled[tgt]['X_ts'][:, :dims].todense()
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    # se crean las rutas para cargar o crear los modelos
    model_name = "%d_%s.pkl" % (dims, tgt)
    model_path = os.path.join(models_path, dataset_name, "indomain", model_name)

    #Se realiza una clasificacion, estimando los parametros mediante cv
    svc = load_best_score(model_path, X_tr, y_tr)
    b_error = 1-svc.score(X_ts, y_ts)

    #############
    #### GFK ####
    #############
    # se adaptan los dominios usando GFK
    print "Adaptando dominios..."
    folder_path = os.path.join(models_path, dataset_name, tipo)
    prefix = "me2_%d_%s_%s.pkl" % (dims, src, tgt)

    best_model_save_path = os.path.join(folder_path, prefix)

    gfk = joblib.load(best_model_save_path)


    X_tr = labeled[src]['X_tr'][:, :dims].todense()
    X_tr_a = transform_gfk(X_tr, gfk)
    y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)

    X_ts = labeled[tgt]['X_ts'][:, :dims].todense()
    X_ts_a = transform_gfk(X_ts, gfk)
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    clf = get_best_score(X_tr_a, y_tr, classifier='SVC', n_jobs=4)
    t_error = 1-clf.score(X_ts_a, y_ts)


    # transfer loss t
    # t_error - b_error
    t_loss = t_error - b_error

    tarea = src[0]+'->'+tgt[0]
    df.loc[i] = ['GFK',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]

    i += 1
            
print "\nPruebas completadas."

In [ ]:
df

In [ ]:
new_scores_path = os.path.join(scores_path,dataset_name, tipo, "me2_%d.csv" % (dims))

print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."

# Pruebas con el dataset Twitter (1000 Dimensiones)

In [ ]:
dims = 1000

In [ ]:
print tipo
print dataset_name
print dims
print data_path

In [ ]:
# cargando dataset Twitter
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

In [ ]:
models_paths = os.path.join(models_path, dataset_name, tipo, 'me2_%d_models_paths.pkl' % dims)

parameters = {
    'dims': [int(dims/4), int(dims/2)],
    'n_subs': [10, 20, 50]
}

pairs = list(itertools.permutations(domains, 2))

#por cada par se entrena un adaptador y se guarda el que mejor resultados entrega
print "Adaptando entre:"
for src, tgt in pairs:
    print "%s - %s" % (src, tgt)
    X_src = labeled[src]['X_tr'][:, :dims].todense()
    y_src = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)

    X_tgt = labeled[tgt]['X_tr'][:, :dims].todense()


    gfk, score = gfk_grid_search(X_src, y_src, X_tgt, parameters, n_jobs=4)
    print "\tMejor modelo: %.3f" % score

    folder_path = os.path.join(models_path, dataset_name, tipo)
    prefix = "me2_%d_%s_%s.pkl" % (dims, src, tgt)

    best_model_save_path = os.path.join(folder_path, prefix)
    print "\tGuardando modelo en %s" % best_model_save_path
    joblib.dump(gfk, best_model_save_path)

print "\nAdaptaciones completadas"

### Pruebas de clasificación

In [ ]:
df = pd.DataFrame(columns=dataframe_columns)

i=0
tareas = len(domains)*(len(domains)-1)
pairs = list(itertools.permutations(domains, 2))

# por cada par posible para adaptar
for src, tgt in pairs:
    print "Tarea %d de %d" % (i+1, tareas)
            
    #baseline in-domain error
    #e_b(T,T)
    #entrenado en dominio tgt y probado en dominio tgt
    X_tr = labeled[tgt]['X_tr'][:, :dims].todense()
    y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)

    X_ts = labeled[tgt]['X_ts'][:, :dims].todense()
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    # se crean las rutas para cargar o crear los modelos
    model_name = "%d_%s.pkl" % (dims, tgt)
    model_path = os.path.join(models_path, dataset_name, "indomain", model_name)

    #Se realiza una clasificacion, estimando los parametros mediante cv
    svc = load_best_score(model_path, X_tr, y_tr)
    b_error = 1-svc.score(X_ts, y_ts)

    #############
    #### GFK ####
    #############
    # se adaptan los dominios usando GFK
    print "Adaptando dominios..."
    folder_path = os.path.join(models_path, dataset_name, tipo)
    prefix = "me2_%d_%s_%s.pkl" % (dims, src, tgt)

    best_model_save_path = os.path.join(folder_path, prefix)

    gfk = joblib.load(best_model_save_path)


    X_tr = labeled[src]['X_tr'][:, :dims].todense()
    X_tr_a = transform_gfk(X_tr, gfk)
    y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)

    X_ts = labeled[tgt]['X_ts'][:, :dims].todense()
    X_ts_a = transform_gfk(X_ts, gfk)
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    clf = get_best_score(X_tr_a, y_tr, classifier='SVC', n_jobs=4)
    t_error = 1-clf.score(X_ts_a, y_ts)


    # transfer loss t
    # t_error - b_error
    t_loss = t_error - b_error

    tarea = src[0]+'->'+tgt[0]
    df.loc[i] = ['GFK',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]

    i += 1
            
print "\nPruebas completadas."

In [ ]:
df

In [ ]:
new_scores_path = os.path.join(scores_path,dataset_name, tipo, "me2_%d.csv" % (dims))

print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."